In [109]:
from __future__ import print_function

from time import time
import logging
import matplotlib.pyplot as plt
import numpy as np
from skimage import color
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA as RandomizedPCA
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical


print(__doc__)

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')


###############################################################################
# Download the data, if not already on disk and load it as numpy arrays

lfw_people = fetch_lfw_people(min_faces_per_person=40, resize=1)

# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape

print(lfw_people.images.shape)
# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.images
n_features = X.shape[1]

# the label to predict is the id of the person
y = lfw_people.target

target_names = lfw_people.target_names
n_classes = target_names.shape[0]
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Automatically created module for IPython interactive environment
(1867, 125, 94)
Total dataset size:
n_samples: 1867
n_features: 125
n_classes: 19


some preprocessing

In [110]:
X_3chan = color.grey2rgb(X)
y = to_categorical(y, num_classes=n_classes)

In [111]:
# from skimage.viewer import ImageViewer
# viewer = ImageViewer(X_3chan[1])
# viewer.show()

Split into test and training sets

In [112]:
X_train, X_test, y_train, y_test = train_test_split(
    X_3chan, y, test_size=0.25)

In [113]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1400, 125, 94, 3)
(1400, 19)
(467, 125, 94, 3)
(467, 19)


Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
dataset): unsupervised feature extraction / dimensionality reduction

In [114]:
# n_components = 150

# print("Extracting the top %d eigenfaces from %d faces"
#       % (n_components, X_train.shape[0]))
# t0 = time()
# pca = RandomizedPCA(n_components=n_components, whiten=True).fit(X_train)
# print("done in %0.3fs" % (time() - t0))

# eigenfaces = pca.components_.reshape((n_components, h, w, 1))
# print(eigenfaces.shape)

Evaluate against pretrained network

In [115]:
base_model_test = InceptionV3(weights='imagenet', include_top=False, input_shape=(h,w,3))

make model

In [116]:

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer
predictions = Dense(n_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

BS = 8

# train the model on the new data for a few epochs
model.fit(X_train, y_train, batch_size=BS, epochs=10)

Epoch 1/10
1400/1400 [==============================] - 36s 26ms/step - loss: 2.7292 - acc: 0.2964
Epoch 2/10
1400/1400 [==============================] - 15s 11ms/step - loss: 2.1210 - acc: 0.3836
Epoch 3/10
1400/1400 [==============================] - 15s 11ms/step - loss: 1.9896 - acc: 0.4350
Epoch 4/10
1400/1400 [==============================] - 15s 11ms/step - loss: 1.8250 - acc: 0.4614
Epoch 5/10
1400/1400 [==============================] - 16s 12ms/step - loss: 1.8173 - acc: 0.4721
Epoch 6/10
1400/1400 [==============================] - 16s 12ms/step - loss: 1.7737 - acc: 0.4857
Epoch 7/10
1400/1400 [==============================] - 15s 11ms/step - loss: 1.7262 - acc: 0.5029
Epoch 8/10
1400/1400 [==============================] - 15s 11ms/step - loss: 1.6519 - acc: 0.5186
Epoch 9/10
1400/1400 [==============================] - 15s 11ms/step - loss: 1.6404 - acc: 0.5157
Epoch 10/10
1400/1400 [==============================] - 15s 11ms/step - loss: 1.6111 - acc: 0.5300


In [107]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(X_train, y_train, batch_size=BS, epochs=10)

0 input_7
1 conv2d_565
2 batch_normalization_565
3 activation_565
4 conv2d_566
5 batch_normalization_566
6 activation_566
7 conv2d_567
8 batch_normalization_567
9 activation_567
10 max_pooling2d_25
11 conv2d_568
12 batch_normalization_568
13 activation_568
14 conv2d_569
15 batch_normalization_569
16 activation_569
17 max_pooling2d_26
18 conv2d_573
19 batch_normalization_573
20 activation_573
21 conv2d_571
22 conv2d_574
23 batch_normalization_571
24 batch_normalization_574
25 activation_571
26 activation_574
27 average_pooling2d_55
28 conv2d_570
29 conv2d_572
30 conv2d_575
31 conv2d_576
32 batch_normalization_570
33 batch_normalization_572
34 batch_normalization_575
35 batch_normalization_576
36 activation_570
37 activation_572
38 activation_575
39 activation_576
40 mixed0
41 conv2d_580
42 batch_normalization_580
43 activation_580
44 conv2d_578
45 conv2d_581
46 batch_normalization_578
47 batch_normalization_581
48 activation_578
49 activation_581
50 average_pooling2d_56
51 conv2d_577
52

In [108]:
model.evaluate(X_test, y_test)


322/322 [==============================] - 12s 37ms/step


[8.859947086120984, 0.4503105590062112]